In [2]:
import torch
from torchvision.models import vgg19
from torchvision import transforms
import torch
from torch.utils.data import Dataset, DataLoader

from PIL import Image

import numpy as np
import torch
import os
from PIL import Image, ImageOps, ImageFilter
import random
import time
import pandas as pd


In [2]:
os.listdir("./Artifact")

['afhq',
 'big_gan',
 'celebahq',
 'cips',
 'coco',
 'cycle_gan',
 'ddpm',
 'denoising_diffusion_gan',
 'diffusion_gan',
 'face_synthetics',
 'ffhq',
 'gansformer',
 'gau_gan',
 'generative_inpainting',
 'glide',
 'imagenet',
 'lama',
 'landscape',
 'latent_diffusion',
 'lsun',
 'mat',
 'metfaces',
 'palette',
 'projected_gan',
 'pro_gan',
 'sfhq',
 'stable_diffusion',
 'star_gan',
 'stylegan1',
 'stylegan2',
 'stylegan3',
 'taming_transformer',
 'vq_diffusion']

In [3]:
dataDir = "./Artifact/"
tsfms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor()
])

In [3]:
trainDataLoader, validDataLoader, testDataLoader = ImageData.getImagesDataLoader(dataDir, batchSize=32, transforms = tsfms)

In [15]:
["sdf"] * 4

['sdf', 'sdf', 'sdf', 'sdf']

In [20]:
class ImageData(Dataset):
    test_generators = ['ddpm', 'gau_gan', 'pallet', 'cips']
    real_data = ['afhq', 'celebahq', 'coco', 'imagenet', 'landscape', 'lsun', 'metfaces', '']

    def __init__(self, dataParentFolder: str, dataIdxs: list = None, transform = None, batch_size=128):
        self.dataParentFolder = dataParentFolder
        self.transform = transform

        imagePaths = []
        imageDataset = []
        fake = []
        for dataset in os.listdir(dataParentFolder):
            path = os.path.join(dataParentFolder, dataset)
            df = pd.read_csv(os.path.join(path, 'metadata.csv'))
            imageFiles = df['image_path'].tolist()
            print(len(imageFiles))
            imagePaths.extend([os.path.join(path, imageFile).replace("\\", "/") for imageFile in imageFiles])
            imageDataset.extend([dataset] * len(imageFiles))
            fake.extend(df["target"])

        self.imagePaths = np.array(imagePaths)
        self.imageDataset = np.array(imageDataset)
        self.fake = np.array(fake)

        if dataIdxs:
            self.imagePaths = self.imagePaths[dataIdxs]
            self.imageDataset = self.imageDataset[dataIdxs]
            self.fake = self.fake[dataIdxs]

    def __len__(self):
        return len(self.imagePaths)

    def __getitem__(self, idx: int):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        imagePath = self.imagePaths[idx]
        pathSplits = imagePath.split(self.dataParentFolder)[1].split("/")
        image = Image.open(imagePath)
        if self.transform:
            image = self.transform(image)

        label = 0
        dataset = self.imageDataset[idx]
        if self.fake[idx] > 0:
            label = self.image_to_vector(dataset)

        return image, label
    
    # TODO: 
    def image_to_vector(self, dataset):
        labels = {
            'big_gan': 1,
            'cips' : 1,
            'cycle_gan' : 1,
            'ddpm' : 2,
            'denoising_diffusion_gan' : 2,
            'diffusion_gan' : 2,
            'face_synthetics' : 3,
            'gansformer' : 1,
            'gau_gan' : 1,
            'generative_inpainting' : 1,
            'glide' : 2,
            'lama' : 3,
            'latent_diffusion' : 2,
            'mat' : 3,
            'palette' : 2,
            'projected_gan' : 1,
            'pro_gan' : 1,
            'sfhq' : 2,
            'stable_diffusion' : 2,
            'star_gan' : 1,
            'stylegan1' : 1,
            'stylegan2' : 1,
            'stylegan3' : 1,
            'taming_transformer' : 3,
            'vq_diffusion' : 2
        }

        return torch.tensor(labels.get(dataset))


In [6]:
# Diffusion-based Generators:

# 'denoising_diffusion_gan'
# 'diffusion_gan'
# 'latent_diffusion'
# 'stable_diffusion'
# 'vq_diffusion'
# 'ddpm' (Diffusion Probabilistic Models)
# GAN-based Generators:

# 'big_gan'
# 'cycle_gan'
# 'gansformer'
# 'gau_gan'
# 'generative_inpainting'
# 'projected_gan'
# 'pro_gan'
# 'star_gan'
# 'stylegan1'
# 'stylegan2'
# 'stylegan3'
# 'taming_transformer'
# Other Generators:

# 'cips'
# 'face_synthetics'
# 'glide'
# 'lama'
# 'mat'
# 'palette'
# 'sfhq'

# gens = [
            'big_gan': 1,
            'cips' : 1,
            'cycle_gan' : 1,
            'ddpm' : 2,
            'denoising_diffusion_gan' : 2,
            'diffusion_gan' : 2,
            'face_synthetics' : 3,
            'gansformer' : 1,
            'gau_gan' : 1,
            'generative_inpainting' : 1,
            'glide' : 2,
            'lama' : 3,
            'latent_diffusion' : 2,
            'mat' : 3,
            'palette' : 2,
            'projected_gan' : 1,
            'pro_gan' : 1,
            'sfhq' : 2,
            'stable_diffusion' : 2,
            'star_gan' : 1,
            'stylegan1' : 1,
            'stylegan2' : 1,
            'stylegan3' : 1,
            'taming_transformer' : 3,
            'vq_diffusion' : 2
# ]
# count = 0
# for thing in gens:
#     print(f"\"{thing}\":{count},")
#     count+=1

In [21]:
trainDatset = ImageData(dataDir, transform=tsfms)

31933
10000
30000
11200
163846
15210
896
10000
15507
10000
70000
10000
7000
22000
20903
96788
24705
4318
20000
539163
60000
1336
6000
12000
40000
10000
21444
9995
10000
1000000
97494
105000
10000


In [22]:
trainDatset.imageDataset

array(['afhq', 'afhq', 'afhq', ..., 'vq_diffusion', 'vq_diffusion',
       'vq_diffusion'], dtype='<U23')

In [7]:
import pandas as pd

In [23]:
testDf = pd.DataFrame(trainDatset.imageDataset)

In [29]:
testDf[testDf[0] == "ddpm"]

,0
262189,ddpm
262190,ddpm
262191,ddpm
262192,ddpm
262193,ddpm
...,...
263080,ddpm
263081,ddpm
263082,ddpm
263083,ddpm


In [34]:
testDf.iloc[262188, 0]

'cycle_gan'

In [35]:
data = trainDatset.__getitem__(262188)

In [36]:
data

(tensor([[[0.2980, 0.3765, 0.3255,  ..., 0.4157, 0.4235, 0.4157],
          [0.1804, 0.2314, 0.2157,  ..., 0.4157, 0.4118, 0.4078],
          [0.1529, 0.2353, 0.2627,  ..., 0.4118, 0.4118, 0.4039],
          ...,
          [0.4235, 0.5255, 0.5176,  ..., 0.8275, 0.7647, 0.5922],
          [0.4196, 0.3725, 0.4471,  ..., 0.8118, 0.6745, 0.6235],
          [0.4902, 0.3020, 0.3333,  ..., 0.8235, 0.7882, 0.8588]],
 
         [[0.3412, 0.4235, 0.3725,  ..., 0.6471, 0.6549, 0.6510],
          [0.2157, 0.2706, 0.2588,  ..., 0.6392, 0.6431, 0.6392],
          [0.1765, 0.2667, 0.2863,  ..., 0.6314, 0.6275, 0.6275],
          ...,
          [0.4275, 0.5255, 0.5176,  ..., 0.8000, 0.7451, 0.5725],
          [0.4157, 0.3647, 0.4392,  ..., 0.7882, 0.6588, 0.6078],
          [0.4824, 0.2941, 0.3216,  ..., 0.8000, 0.7725, 0.8431]],
 
         [[0.2745, 0.3451, 0.2941,  ..., 0.8745, 0.8902, 0.8863],
          [0.1490, 0.1922, 0.1686,  ..., 0.8706, 0.8784, 0.8745],
          [0.1216, 0.1961, 0.1922,  ...,

In [41]:
trainDataLoader = DataLoader(trainDatset, batch_size=8)

In [42]:
for thing in trainDataLoader:
    print(thing)
    break

[tensor([[[[0.9059, 0.9020, 0.8902,  ..., 0.5333, 0.5608, 0.5922],
          [0.9020, 0.9020, 0.8941,  ..., 0.5647, 0.5725, 0.5843],
          [0.8941, 0.8980, 0.8941,  ..., 0.5882, 0.5843, 0.5804],
          ...,
          [0.6706, 0.6588, 0.6471,  ..., 0.1490, 0.1647, 0.1725],
          [0.6784, 0.6627, 0.6471,  ..., 0.1490, 0.1647, 0.1765],
          [0.6745, 0.6588, 0.6392,  ..., 0.1529, 0.1686, 0.1804]],

         [[0.9255, 0.9216, 0.9176,  ..., 0.5647, 0.5882, 0.6196],
          [0.9216, 0.9216, 0.9216,  ..., 0.6000, 0.6000, 0.6118],
          [0.9216, 0.9255, 0.9255,  ..., 0.6235, 0.6118, 0.6078],
          ...,
          [0.7333, 0.7176, 0.6980,  ..., 0.1020, 0.1059, 0.1137],
          [0.7373, 0.7216, 0.7020,  ..., 0.1020, 0.1059, 0.1137],
          [0.7333, 0.7176, 0.6941,  ..., 0.1020, 0.1059, 0.1176]],

         [[0.9373, 0.9333, 0.9294,  ..., 0.5922, 0.6118, 0.6431],
          [0.9333, 0.9333, 0.9294,  ..., 0.6275, 0.6314, 0.6353],
          [0.9294, 0.9333, 0.9333,  ..., 

In [37]:
trainDatset.__getitem__(2000000)[1]

tensor(29)

In [4]:
for thing in trainDataLoader:
    print(thing)
    break

[tensor([[[[0.0431, 0.0510, 0.0549,  ..., 0.0824, 0.0824, 0.0784],
          [0.0431, 0.0510, 0.0549,  ..., 0.0902, 0.0902, 0.0902],
          [0.0431, 0.0510, 0.0510,  ..., 0.0941, 0.0902, 0.0863],
          ...,
          [0.9882, 0.9882, 0.9882,  ..., 0.9961, 0.9961, 0.9961],
          [0.9882, 0.9882, 0.9882,  ..., 0.9961, 0.9961, 0.9961],
          [0.9882, 0.9882, 0.9882,  ..., 0.9961, 0.9961, 0.9961]],

         [[0.0392, 0.0471, 0.0510,  ..., 0.0941, 0.0902, 0.0902],
          [0.0392, 0.0471, 0.0510,  ..., 0.1020, 0.0980, 0.1020],
          [0.0392, 0.0471, 0.0471,  ..., 0.1059, 0.1020, 0.1020],
          ...,
          [0.9725, 0.9725, 0.9725,  ..., 0.9804, 0.9804, 0.9804],
          [0.9725, 0.9725, 0.9725,  ..., 0.9804, 0.9804, 0.9804],
          [0.9725, 0.9725, 0.9725,  ..., 0.9804, 0.9804, 0.9804]],

         [[0.0706, 0.0784, 0.0824,  ..., 0.1412, 0.1373, 0.1255],
          [0.0706, 0.0784, 0.0824,  ..., 0.1490, 0.1451, 0.1373],
          [0.0706, 0.0784, 0.0784,  ..., 

In [47]:
thing[1]

tensor([0, 0, 0, 0, 0, 0, 0, 0])